In [67]:
import os
import time # необходим для использования задержек методом sleep (в коде нет)
from pprint import pprint
import pandas as pd
import requests
import datetime
import json
from dotenv import load_dotenv

load_dotenv()

True

In [82]:
class VkApiHandler:

    base_url = 'https://api.vk.com/method/'

    def __init__(self, access_token, id, version='5.131'):
        self.params = {
            'access_token': access_token,
            'v': version
        }
        self.id = id

    def find_photo_max_size(self, photos_dict):
        """Метод возвращает ссылку на фото максимального размера"""
        photo_max_size = 0
        max_dpi = 0
        types = 'smxopqryzw'
        max_type_index = 0
        for i in range(len(photos_dict)):
            photo_dpi = photos_dict[i].get('width') * photos_dict[i].get('height')
            photo_type = photos_dict[i].get('type')
            if photo_dpi != 0:
                if photo_dpi > max_dpi:
                    max_dpi = photo_dpi
                    photo_max_size = i
            else:
                if types.find(photo_type) > max_type_index:
                    photo_max_size = i
        return photos_dict[photo_max_size].get('url'), photos_dict[photo_max_size].get('type')
    
    def date_time_convert(self, time_unix):
        """Функция преобразует дату загрузки фото в привычный формат"""
        time_conv = datetime.datetime.fromtimestamp(time_unix)
        str_time = time_conv.strftime('%Y-%m-%d_%H-%M-%S')
        return str_time

    def get_photo_info(self):
        """Метод для получения количества фотографий и массива фотографий"""
        url = self.base_url + 'photos.get'
        params = {
            'album_id': 'profile',
            'extended': 1,
            'photo_sizes': 1,
            'rev': 1,
            'owner_id': self.id,
            **self.params
        }
        response = requests.get(url, params=params)
        data = response.json()['response']
        return data['count'], data['items']
    
    def get_photo_params(self):
        """Метод возвращает словарь с параметрами фотографий"""
        photo_count, photo_items = self.get_photo_info()
        result = {}
        for i in range(photo_count):
            number_of_likes = photo_items[i]['likes']['count']
            url_photo, photo_type = self.find_photo_max_size(photo_items[i]['sizes'])
            date_photo = self.date_time_convert(photo_items[i]['date'])
            need_item = result.get(number_of_likes, [])
            need_item.append({
                'number_of_likes': number_of_likes,
                'add_name': date_photo,
                'url_photo': url_photo,
                'size': photo_type
            })
            result[number_of_likes] = need_item            
        return result

    def get_photo_json(self):
        """Возвращает словарь с параметрами фотографий и список JSON"""
        json_list = []
        upload_dict = {}
        photo_dict = self.get_photo_params()
        for el in  photo_dict.keys():
            counter = 0
            for value in photo_dict[el]:
                if len(photo_dict[el]) == 1:
                    file_name = f'{value["number_of_likes"]}.jpeg'
                    upload_dict[file_name] = photo_dict[el][0]['url_photo']
                else:
                    file_name = f'{value["number_of_likes"]}_{value["add_name"]}.jpeg'
                    upload_dict[file_name] = photo_dict[el][counter]['url_photo']
                    counter += 1
                json_list.append({'file_name': file_name, 'size':value['size']})                
        return json_list, upload_dict
 

In [83]:
if __name__ == '__main__':
    vk_token = os.getenv('VK_API_TOKEN')
    vk= VkApiHandler(vk_token, '106420704')

    # with open('vk_photo.json', 'w') as outfile:
    #     json.dump(vk.get_photo_info('1'), outfile)
    
    pprint(vk.get_photo_info())
    # photo_count, photo_items = vk.get_photo_info()
    # for i in range(photo_count):
    #     url, p_size = vk.find_photo_max_size(photo_items[i]['sizes'])
    #     print(f'{p_size}: {url}')
    # pprint(vk.get_photo_params())
    # jl, ud = vk.get_photo_json()
    # pprint(jl)
    # pprint(ud)
    



(28,
 {'count': 28,
  'items': [{'album_id': -6,
             'can_comment': 0,
             'comments': {'count': 0},
             'date': 1650453800,
             'has_tags': False,
             'id': 457240951,
             'likes': {'count': 6, 'user_likes': 0},
             'owner_id': 106420704,
             'reposts': {'count': 0},
             'sizes': [{'height': 75,
                        'type': 's',
                        'url': 'https://sun9-51.userapi.com/impg/eeQOfGg_cPeO3T0Xltfa2EeEOXiETgVDN8rkzA/cAw5ij-tKEc.jpg?size=42x75&quality=95&sign=b41f3bfe840078887c0caf90d2b1a0f7&c_uniq_tag=Pn6FkNhVHbBfiZb1J-uS-Tw82dDOh79E2T0qR8a9bgI&type=album',
                        'width': 42},
                       {'height': 130,
                        'type': 'm',
                        'url': 'https://sun9-51.userapi.com/impg/eeQOfGg_cPeO3T0Xltfa2EeEOXiETgVDN8rkzA/cAw5ij-tKEc.jpg?size=73x130&quality=95&sign=584b897a269667caffd9d6a6f87b0d4c&c_uniq_tag=mNFlzK49yyfxG8gxT2AaX7mhewdkIS